# NAXIVA Analysis - Dynamic Model

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, precision_recall_curve, auc
from sklearn.model_selection import LeaveOneOut, GridSearchCV

import warnings
from warnings import filterwarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
filterwarnings('ignore')


## Dynamic model (all baseline features + Wk3/Wk1 growth factor fold changes)

In [2]:
baseline_data = pd.read_csv('NAXIVA_data/NAXIVA_baseline_data.csv', index_col=0)
dynamic_data = pd.read_csv('NAXIVA_data/NAXIVA_dynamic_data.csv', index_col=0)

mod2_data = pd.merge(baseline_data, dynamic_data, left_index=True, right_index=True)


responders = pd.read_csv('NAXIVA_data/NAXIVA_response_labels.csv')


# Impute the missing patient data
imputer = IterativeImputer(random_state = 42, keep_empty_features = False)
mod2_imputed = imputer.fit_transform(mod2_data)
mod2_data_imputed = pd.DataFrame(mod2_imputed, columns = mod2_data.columns, index = mod2_data.index)
# mod2_data_imputed.replace(to_replace = 0, value = 1, inplace = True)

# Scale the data using Min-Max scaling
sc = MinMaxScaler((0, 1)) 
mod2_data_norm = sc.fit_transform(mod2_data_imputed)
mod2_data_scaled = pd.DataFrame(mod2_data_norm, columns = mod2_data_imputed.columns, index = mod2_data_imputed.index)
mod2_data_final = mod2_data_scaled

# mod2_data_final.to_csv('data/mod2_data_final.csv')

# Curate list of responders and replace words with numbers
responders = responders.replace(["non-responder", "stopped_early", "responder"], [0, 0, 1])
responders.set_index('trial_id', inplace = True)

In [3]:
def remove_correlated_features(df, threshold):
    corr_matrix = df.corr(method='spearman').abs() # create correlation matrix of absolute correlation coefficient
    iters = range(len(corr_matrix.columns) - 1) # calculate how many iterations are needed
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i+1):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)] # iterate over upper triangle of the correlation matrix
            col = item.columns
            row = item.index
            val = abs(item.values)

            # If correlation exceeds the threshold
            if val >= threshold:
                print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2)) # Print the correlated features and the correlation value
                drop_cols.append(col.values[0])

    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    print(df.shape)
    print(f'Columns dropped: {drops}')
    print(f'N of columns dropped: {len(drops)}')
    df = df.drop(columns=drops)
    print(df.shape)
    return df

In [4]:
# Define threshold
threshold = 0.75
# Remove highly correlated features from database
mod2_data_final = remove_correlated_features(mod2_data_final, threshold)
print(f'Features remaining: {list(mod2_data_final.columns)}')

CCL2 baseline | IL-8 baseline | 0.88
VEGF-C baseline | CCL13 baseline | 0.83
CD34+ area baseline | CD31+ area baseline | 0.94
CD31+ CD34+ area baseline | CD31+ area baseline | 0.96
CD31+ CD34+ area baseline | CD34+ area baseline | 0.98
CD8+ PD-1+ baseline | GZMB+ baseline | 0.83
CD8+ GZMB- PD-1+ baseline | CD8+ baseline | 0.82
CD8+ GZMB- PD-1+ baseline | CD8+ PD-1+ baseline | 0.97
CD8+ GZMB+ PD-1+ baseline | GZMB+ baseline | 0.91
CD8+ GZMB+ PD-1+ baseline | CD8+ PD-1+ baseline | 0.88
CD8+ GZMB+ PD-1+ baseline | CD8+ GZMB- PD-1+ baseline | 0.82
FOXP3+ baseline | CD4+ baseline | 0.86
PD-1+ baseline | CD4+ baseline | 0.91
PD-1+ baseline | FOXP3+ baseline | 0.91
Treg baseline | CD4+ baseline | 0.9
Treg baseline | FOXP3+ baseline | 0.97
Treg baseline | PD-1+ baseline | 0.95
Treg PD-1+ baseline | CD4+ baseline | 0.91
Treg PD-1+ baseline | FOXP3+ baseline | 0.9
Treg PD-1+ baseline | PD-1+ baseline | 0.95
Treg PD-1+ baseline | Treg baseline | 0.94
Treg PD-1- baseline | CD4+ baseline | 0.85
Tre

### Leave one-out supervised learning

In [5]:
mod2_methods = {
    'Logistic Regression RFE 3': RFE(estimator = LogisticRegression(), n_features_to_select=3, step=1),
}

# Choose supervised learning methods
mod2_algorithms = {
    "Logistic SGD" : (SGDClassifier(loss = 'log_loss', class_weight='balanced', max_iter=1000), {
        'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'learning_rate': ['optimal', 'invscaling', 'adaptive'],
        'eta0': [0.01, 0.1, 1],
    }),
}

In [6]:
# Make multi-index data frame to save results
mod2_method_names = list(mod2_methods.keys())
mod2_alg_names = list(mod2_algorithms.keys())
iterations = range(20)
mod2_index = pd.MultiIndex.from_product([mod2_methods, mod2_algorithms, iterations], names=['Method', 'Algorithm', 'Iteration'])
mod2_results = pd.DataFrame(index=mod2_index, columns=['trial_id', 'hyperparams', 'selected_features', 'coef', 'auc', 'accuracy', 'sensitivity', 'precision', 'f1-score', 'auprc', 'scores'])

# Generate empty series for outputs: prediction and score
y_true = responders['responder']
y_pred = pd.Series(index = y_true.index)
y_score = pd.Series(index = y_true.index)

# Create a series called y_train_pred for each training set in the leave-one-out cross-validation
y_train_pred = pd.DataFrame(index = mod2_index, columns=responders.index)
mod2_results = pd.merge(mod2_results, y_train_pred, left_index=True, right_index=True, how = 'inner')

random.seed(10)
np.random.seed(10)

for method_name in mod2_methods: # For each method in the pre-formed list of methods
    method = mod2_methods[method_name]

    for alg_name, (alg, param_grid_2) in mod2_algorithms.items(): # Within each method, loop over all the algorithms in the pre-formed list of algorithms

        random.seed(10)
        np.random.seed(10)

        # Generate leave-one-out index lists
        loo = LeaveOneOut() 
        for i, (train_idx, test_idx) in enumerate(loo.split(y_true)): # Create lists of indices which leaves one patient out with each iteration
            y_train = y_true[train_idx]
            y_test = y_true[test_idx]
            X_train = mod2_data_final.loc[list(y_train.index)]
            X_test = mod2_data_final.loc[list(y_test.index)]

            # Apply feature selection algorithm for each leave-one-out iteration
            data = method.fit(X_train, y_train)
            mask = method.get_support(indices = True)
            selected_features = list(X_train.iloc[:, mask])

            mod2_results.at[(method_name, alg_name, i), 'selected_features'] = (selected_features) # Save feature names to dataframe               

            # Apply only selected features from the cytokine dataset for each leave-one-out iteration
            X_train = X_train[selected_features]
            X_test = X_test[selected_features]

            # Hyperparameter tuning
            selected_hyperparameters_2 = []
            grid_search = GridSearchCV(alg, param_grid = param_grid_2, cv = 5, scoring = 'roc_auc') # Param grid is specified earlier for each algorithm
            grid_search.fit(X_train, y_train)
            best_params = grid_search.best_params_
            selected_hyperparameters_2 = best_params # Save selected hyperparameters for each split
            alg.set_params(**best_params) # Set best hyperparameters for algorithm    

            # Train and predict
            alg.fit(X_train, y_train)  # Fit the algorithm on the training set

            # Save coefficients for logistic regression
            coef_dict = {}
            for feature, coef in zip(alg.feature_names_in_, alg.coef_[0]):
                coef_dict[feature] = coef
            mod2_results.loc[(method_name, alg_name, i), 'coef'] = [(coef_dict)] # Save selected features coefficients

            # Save predicted label to y_pred series
            y_pred[list(y_test.index)] = alg.predict(X_test) 
            y_score[list(y_test.index)] = alg.predict_proba(X_test)[:, 1]
            mod2_results.at[(method_name, alg_name, i), 'trial_id'] = y_test.index[0]
            mod2_results.at[(method_name, alg_name, i), 'scores'] = alg.predict_proba(X_test)[:,1]
            
            for j, k in enumerate(X_train.index):
                mod2_results.at[(method_name, alg_name, i), k] = alg.predict_proba(X_train)[:,1][j]

            mod2_results.loc[(method_name, alg_name, i), 'hyperparams'] = [selected_hyperparameters_2.copy()] # Save selected hyperparameters

        precision, recall, thresholds = precision_recall_curve(y_true, y_score)
        auprc = auc(recall, precision)

        mod2_results.loc[(method_name, alg_name, 0), 'auc'] = roc_auc_score(y_true, y_score) # Save AUC ROC score
        mod2_results.loc[(method_name, alg_name, 0), 'accuracy'] = accuracy_score(y_true, y_pred) # Save accuracy score
        mod2_results.loc[(method_name, alg_name, 0), 'sensitivity'] = recall_score(y_true, y_pred) # Save sensitivity score
        mod2_results.loc[(method_name, alg_name, 0), 'precision'] = precision_score(y_true, y_pred) # Save precision score
        mod2_results.loc[(method_name, alg_name, 0), 'f1-score'] = f1_score(y_true, y_pred) # Save f1-score
        mod2_results.loc[(method_name, alg_name, 0), 'auprc'] = auc(recall, precision) # Save AUC PRC score
        # mod2_results.loc[(method_name, alg_name, 0), 'scores'] = [y_score.copy()] # Save scores
        
        # Check LOOCV 
        print(method_name)
        print(alg_name)
        print(f'Patients: {len(list(y_true.index))}')
        print(f'Predictions: {len(list(y_pred.index))}')
        print(f'Iterations: {i+1}')
        print(f'AUC: {roc_auc_score(y_true, y_score)}')
        assert len(list(y_true.index)) == len(list(y_pred.index)), 'True and predicted labels have different lengths'


mod2_results.to_csv(f"dynamic_model_results.csv")

Logistic Regression RFE 3
Logistic SGD
Patients: 20
Predictions: 20
Iterations: 20
AUC: 0.945054945054945
Youden's index: 0.49739721924428687
